In [4]:
## Sequence Analysis
   # Import library and load files

In [5]:
import pandas as pd 

In [6]:
df= pd.read_csv("GA4_synthetic_data.csv")

In [7]:
df.head()

,event_date,event_name,user_id,session_id,device_category,country,page_location,engagement_time_msec,item_revenue,traffic_source
0,20240101,click,84327,749914,tablet,Canada,/about,20062,260.48,organic
1,20240101,purchase,97911,910692,desktop,UK,/about,20648,276.12,direct
2,20240101,session_start,98147,814967,mobile,USA,/about,4751,43.17,paid
3,20240101,purchase,17833,161852,desktop,Canada,/contact,3056,203.71,paid
4,20240101,purchase,57013,128380,tablet,USA,/checkout,38649,259.89,paid


In [8]:
## Convert event_date to Real Date (Very Important)

In [9]:
df["event_date"]=pd.to_datetime(df["event_date"], format="%Y%m%d")

In [10]:
df.head()

,event_date,event_name,user_id,session_id,device_category,country,page_location,engagement_time_msec,item_revenue,traffic_source
0,2024-01-01,click,84327,749914,tablet,Canada,/about,20062,260.48,organic
1,2024-01-01,purchase,97911,910692,desktop,UK,/about,20648,276.12,direct
2,2024-01-01,session_start,98147,814967,mobile,USA,/about,4751,43.17,paid
3,2024-01-01,purchase,17833,161852,desktop,Canada,/contact,3056,203.71,paid
4,2024-01-01,purchase,57013,128380,tablet,USA,/checkout,38649,259.89,paid


In [11]:
## Sort Data in Correct Order (SUPER IMPORTANT)

In [12]:
df= df.sort_values(by=["user_id","session_id","engagement_time_msec"])

In [13]:
df.head()

,event_date,event_name,user_id,session_id,device_category,country,page_location,engagement_time_msec,item_revenue,traffic_source
335,2024-01-01,session_start,10002,967595,tablet,UK,/home,14680,482.91,organic
5121,2024-01-04,click,10005,553498,tablet,USA,/about,38617,123.66,referral
3798,2024-01-03,scroll,10015,834781,desktop,Canada,/checkout,41193,349.97,organic
6162,2024-01-05,page_view,10020,394319,tablet,India,/home,34780,302.35,paid
4751,2024-01-04,add_to_cart,10022,206680,mobile,Canada,/home,31761,53.25,organic


In [14]:
## Create Event Sequences Per Session 

In [17]:
session_sequence= df.groupby("session_id")["event_name"].apply(list).reset_index()

In [18]:
session_sequence.head()

,session_id,event_name
0,100056,[purchase]
1,100068,[session_start]
2,100184,[scroll]
3,100189,[add_to_cart]
4,100318,[session_start]


In [19]:
## Make Sequence Text (Human-Friendly Format)-- optionla

In [22]:
session_sequence["sequence_str"] = session_sequence["event_name"].apply(lambda x: " → ".join(x))

In [23]:
session_sequence.head()

,session_id,event_name,sequence_str
0,100056,[purchase],purchase
1,100068,[session_start],session_start
2,100184,[scroll],scroll
3,100189,[add_to_cart],add_to_cart
4,100318,[session_start],session_start


In [24]:
### FUNNEL ANALYSIS
        ## Out of 100 users who start something, how many reach the end? Where do users drop off?

In [25]:
## CREATE EVENT FLAGS

In [28]:
funnel_steps = [
    "session_start",
    "page_view",
    "add_to_cart",
    "purchase"
]

In [29]:
funnel_df= (df.groupby ("session_id")["event_name"].apply(list).reset_index())
for step in funnel_steps:
    funnel_df[step]=funnel_df["event_name"].apply(lambda x: 1 if step in x else 0)

In [32]:
funnel_df.head(20)

,session_id,event_name,session_start,page_view,add_to_cart,purchase
0,100056,[purchase],0,0,0,1
1,100068,[session_start],1,0,0,0
2,100184,[scroll],0,0,0,0
3,100189,[add_to_cart],0,0,1,0
4,100318,[session_start],1,0,0,0
5,100415,[session_start],1,0,0,0
6,100599,[add_to_cart],0,0,1,0
7,100637,[add_to_cart],0,0,1,0
8,100638,[click],0,0,0,0
9,100814,[purchase],0,0,0,1


In [33]:
# CALCULATE DROP-OFFS

In [34]:
funnel_counts = { step: funnel_df[step].sum()
                 for step in funnel_steps}


In [35]:
funnel_counts

{'session_start': np.int64(1622),
 'page_view': np.int64(1664),
 'add_to_cart': np.int64(1657),
 'purchase': np.int64(1689)}

In [36]:
## STEP 4: TOP DROP-OFF POINT (Top steps causing loss)

In [47]:
dropoff_data = []

for i in range(len(funnel_steps) - 1):
    current_step = funnel_steps[i]
    next_step = funnel_steps[i + 1]

    dropped_sessions = funnel_df[
        (funnel_df[current_step] == 1) &
        (funnel_df[next_step] == 0)
    ]

    dropoff_data.append({
        "From Step": current_step,
        "To Step": next_step,
        "Users Dropped": dropped_sessions.shape[0]
    })

# ----------------------------------------
# STEP 5: Create final drop-off table
# ----------------------------------------
dropoff_df = pd.DataFrame(dropoff_data)

# ----------------------------------------
# STEP 6: Sort to find TOP drop-off points
# ----------------------------------------
dropoff_df = dropoff_df.sort_values(
    by="Users Dropped",
    ascending=False
)

dropoff_df

,From Step,To Step,Users Dropped
1,page_view,add_to_cart,1658
2,add_to_cart,purchase,1654
0,session_start,page_view,1622


In [48]:
### TIME-BASED FUNNEL ANALYSIS

In [49]:
# CREATE SESSION-LEVEL TOTAL TIME

In [50]:
session_time=(df.groupby("session_id")["engagement_time_msec"].sum().reset_index())

In [51]:
session_time.head()

,session_id,engagement_time_msec
0,100056,49684
1,100068,13118
2,100184,43424
3,100189,44904
4,100318,16351


In [52]:
# MERGE TIME WITH FUNNEL DATA

In [55]:
funnel_time_df= funnel_df.merge(session_time, on ="session_id", how="left") 
# Convert milliseconds to seconds (for easy understanding)
funnel_time_df["engagement_time_sec"] = funnel_time_df["engagement_time_msec"] / 1000


In [56]:
funnel_time_df.head()

,session_id,event_name,session_start,page_view,add_to_cart,purchase,engagement_time_msec,engagement_time_sec
0,100056,[purchase],0,0,0,1,49684,49.684
1,100068,[session_start],1,0,0,0,13118,13.118
2,100184,[scroll],0,0,0,0,43424,43.424
3,100189,[add_to_cart],0,0,1,0,44904,44.904
4,100318,[session_start],1,0,0,0,16351,16.351


In [58]:
## STEP 4: COMPARE TIME — CONVERTED vs DROPPED USERS
              # “Do users who convert behave differently (time-wise) than users who drop?”
              #  If yes → time is a key signal for drop-off & conversion.

In [59]:
# Separate converters & non-converters

In [60]:
converted= funnel_time_df[funnel_time_df["purchase"]==1]
dropped = funnel_time_df[funnel_time_df["purchase"] == 0]

In [64]:
# TIME BUCKET ANALYSIS

In [65]:
# Define time buckets (in seconds)

In [69]:
bins= [0,10,30,60,120,300]
labels= ["0-10s", "10-30s", "30-60s", "60-120s","120s+"]

funnel_time_df["time_bucket"]=pd.cut( funnel_time_df["engagement_time_sec"], bins=bins,labels=labels,include_lowest=True)


In [70]:
funnel_time_df[["engagement_time_sec", "time_bucket"]].head()

,engagement_time_sec,time_bucket
0,49.684,30-60s
1,13.118,10-30s
2,43.424,30-60s
3,44.904,30-60s
4,16.351,10-30s


In [71]:
# Time Bucket vs Conversion

In [74]:
# Create conversion status column
funnel_time_df["conversion_status"] = funnel_time_df["purchase"].apply(
    lambda x: "Converted" if x == 1 else "Dropped"
)

# Check result
funnel_time_df[["session_id", "purchase", "conversion_status"]].head()

,session_id,purchase,conversion_status
0,100056,1,Converted
1,100068,0,Dropped
2,100184,0,Dropped
3,100189,0,Dropped
4,100318,0,Dropped


In [3]:
bucket_summary=(funnel_time_df.groupby(["time_bucket","conversion_status"]).size().reset_index(name="user"))

NameError: name 'funnel_time_df' is not defined

In [76]:
bucket_summary

,time_bucket,conversion_status,user
0,0-10s,Converted,361
1,0-10s,Dropped,1641
2,10-30s,Converted,657
3,10-30s,Dropped,3343
4,30-60s,Converted,666
5,30-60s,Dropped,3273
6,60-120s,Converted,5
7,60-120s,Dropped,7
8,120s+,Converted,0
9,120s+,Dropped,0
